# Module 3 - Optimizing Python 
----------------------------------------------


In [ ]:
%load_ext autoreload
%autoreload 2

<br>

# Table of Content <a id='toc'></a>

0. [meet the code](#0)

1. [measuring](#1)

    1.1. [timing](#2)
 
    1.2. [measuring RAM usage](#3)


2. [optimize](#4)

   2.1. [Cython](#5)
   
   2.2. [Numba](#6)


3. [Working with processes/threads](#7)

   3.1 [Multiprocessing (and refactoring)](#8)

   3.2. [numba and parallelization](#9)

<br>
<br>


# 0. Meet the code <a id='0'></a>
----------------------------------------------------------

In this notebook we will mostly focus on a simple function which computes pairwise distance between a set of vectors, a very classical operation present in many data analysis methods.

In [ ]:
def pairwise_distance(X):

    num_vectors = len(X)
    num_measurements = len(X[0])
    D = [[0]*num_vectors for x in range(num_vectors)]
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = []
            for k in range(num_measurements):
                d.append( ( X[i][k] - X[j][k] )**2 )
            
            D[i][j] = sum(d) **0.5
    return(D)

<br>
<br>


# 1. Measuring time and memory usage <a id='1'></a>
----------------------------------------------------------

The first step of any code optimization process should be measuring what your code is doing, in order to pinpoint where your effort should be focused.

Here is an example function which computes pairwise Euclidian distances between multiple vectors values.

In [ ]:
import numpy as np

## let's generate some random data to play with : 200x100 matrix
num_vector = 200
num_measures = 100

data = np.random.uniform(size=(num_vector,num_measures))
print(data)
print(data.shape)

<br>
<br>

[back to the toc](#toc)

## 1.1. Measuring time usage <a id='2'></a>

On you terminal, you may measure up the taime taken by a python script execution using:

`time python myscript.py`

Or, using jupyter/ipython magic :

In [ ]:
%time D = pairwise_distance(data)

Or : 

In [ ]:
%%time 
## applies on the whole cell

D = pairwise_distance(data)


That is all nice, but there is always a bit a variability, which becomes very apparent on smaller time.

In [ ]:
# smaller data set
num_vector = 100
num_measures = 10

data = np.random.uniform(size=(num_vector,num_measures))

%time D = pairwise_distance(data)

For this, we have `timeit`:

`python -m timeit myScript.py`

Or :

In [ ]:
%timeit D = pairwise_distance(data)

In [ ]:
# you can specify the number of runs and loops performed
%timeit -n 2 -r 10 D = pairwise_distance(data)

Performs `-r` runs, each time taking the best execution time out of `-n` loops.

Ok, already this is nice, we will definitely be using this to compare together different implementations.

For example, I have rewritten the function using numpy :

In [ ]:

def pairwise_distance_numpy(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    D = np.empty((num_vectors, num_vectors), dtype=np.float64)
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = np.square( np.subtract(X[i], X[j]) )
            D[i, j] = np.sqrt(np.sum(d))
    return(D)

In [ ]:
# 100x100 data set
num_vector = 100
num_measures = 100

data = np.random.uniform(size=(num_vector,num_measures))

print('native:')
%timeit -n 5 -r 3 D = pairwise_distance(data)
print('numpy:')
%timeit -n 5 -r 3 D = pairwise_distance_numpy(data)

Nice! 

<br> 

However, most of the time your code is more complex than a single function, and before optimizing you first want to see which function you should optimize. That is when **profiling** comes in handy.

Imagine you have a script, implementing a Kmeans algorithm. 
Here are the functions which look like the best candidates for optimization :
* `computeDistanceToCentroid` : compute the distance between a poitnand a centroid
* `computeNearestCentroid` : compute which centroid is the closest to each point (actualy calls `computeDistanceToCentroid`, but also possess some other potentially costly computations)
* `computeCentroids` : computes the position of the centroid of a points with a given cluster assignment

Are they really the best candidate ? which one shuld we go for first ?

We recommend [cProfile](https://docs.python.org/3/library/profile.html)

in the terminal :

```
python -m cProfile -o profile.log -s cumtime myScript.py
```

will execute the script, and profile time usage of every functions
 * `-o` : output file for the profiling log
 * `-s cumtime` : to sort by cumulative time spent in a single function

in jupyter :

In [ ]:
# generating some random data 
def generateCluster( n , means , sds ):
    P = np.random.randn( len(means) , n )
    for i in range(len(means)):
        P[i,] = P[i,] * sds[i] + means[i]
    
    return P


clusterSizes = [4000,2000,4000,4000,2000 ]
clusterMeans = [ [ 0 , -2 ] , [ 3 , 3 ] ,[ -1 , 3 ], [-5, 0] , [5,-1] ]
clusterSDs = [ [0.5,1] ,[1,0.5] ,[0.5,0.5],[2,1] ,[1,1] ]
C = []
A = []
for i in range( len(clusterSizes) ):
    C.append( generateCluster( clusterSizes[i] , clusterMeans[i] , clusterSDs[i] ) )
    A += [i]*clusterSizes[i]
Points = np.concatenate( C , axis=1)
realAssignment = np.array(A)


In [ ]:
from Kmeans import Kmeans
# performing Kmean
k=5
%prun -l 30 -s cumtime  kmeanAssignment = Kmeans( Points , k , maxNbRounds=1000 ) 
# the %prun magic command activate profiling
#  -l 30 : limits the report to 30 lines
#  -s cumtime : sort by decreasing cumtime


The columns correspond to:
 * ncalls : for the number of calls.
 * tottime : for the total time spent in the given function (and excluding time made in calls to sub-functions)
 * percall : is the quotient of tottime divided by ncalls
 * cumtime : is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
 * percall : is the quotient of cumtime divided by primitive calls


### Micro-Exercise:
* Look at the profile. Where should optimization efforts go first ?

<br>
<br>

[back to the toc](#toc)

## 1.2. measuring RAM usage <a id='3'></a>

Sometimes, you also want to measure the memory imprint of your code 

The nicest tool I know about for that is [memory-profiler](https://pypi.org/project/memory-profiler/)

Install it with :

```python
!pip install memory_profiler
```

Basically, in your code you add a **decorator** to the function of interest:

In [ ]:
from memory_profiler import profile

@profile
def my_func():
    a = [1] * (10 ** 6)
    b = [2] * (2 * 10 ** 7)
    del b
    return a

Then, you may either do it command-line style  :

```
python -m memory_profiler example.py
```

or prefer jupyter-magic:

In [ ]:
%load_ext memory_profiler

In [ ]:
%memit my_func()

It gives us a result: `peak memory: 210.19 MiB, increment: 152.58 MiB`
            
But it does not like that our code is in the same notebook. Let's have it in another script :

In [ ]:
from utils import my_func

%memit my_func()

Now, that is quite nice : we have a run down of the RAM usage, line-by-line.

### Micro-Exercise:
* profile the memory usage of `pairwise_distance`

<br>
<br>
<br>

[back to the toc](#toc)

# 2. Code optimization <a id='4'></a>
---------------------------------

Now that we have seen the tools to measure our code resource usage, we will review a couple of tricks that can help you speedup your python code tremendously.

The firsts are basic:
 1. **apply standard good-sense** : does your code reads/write to the disk more than it need to ? Do you spend a lot of time searching for items in lists instead of dictionnaries ?
 2. **switch to numpy** : vectorized operations are great (as we have seen). 
 
> don't know how to numpy? You can [get started here](https://numpy.org/doc/stable/user/quickstart.html)


### Let's generate a toy dataset of random number vectors

In [ ]:
num_vector = 200
num_measures = 100

data = np.random.uniform(size=(num_vector,num_measures))
print(type(data[0][0]))
print(data.shape)

### Pure Python (and numpy)

In [ ]:
%timeit result = pairwise_distance_numpy(data)

<br>
<br>

[back to the toc](#toc)

## 2.1. Cython <a id='5'></a>

**[Cython](https://cython.org/)** provides way to transform a python code into C compiled code failry seamlessly.

By default, Cython retains Python flexibility by creating the ugliest of C-codes. This comes at the cost of a lot of efficiency, but already it is enough to speed your code some.

The "command-line" flavor of cython involves either calling `cython` or writing a little `setup.py` file for your code. It is a bit of work at the start but actually quite easy once you have done it a couple of time : see [here for examples](https://cython.readthedocs.io/en/latest/src/quickstart/build.html)

The jupyter way :

In [ ]:
%load_ext cython

In [ ]:
## pure python 
def f_native(x):
    return x ** 2 - x


def integrate_f_native(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_native(a + i * dx)
    return s * dx

In [ ]:
%%cython
## cython, without changing a single thing

def f(x):
    return x ** 2 - x


def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [ ]:
print("native")
%timeit -n 3 -r 5 result = integrate_f_native(0,1,1000000)
print("simple cython")
%timeit -n 3 -r 5 result = integrate_f(0,1,1000000)

Ok, so a speedup of about a third, fairly nice for a single line change.

But, let's look how Cython performed with our code :

In [ ]:
%%cython --annotate
## cython, without changing a single thing

def f(x):
    return x ** 2 - x


def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

We can give some hints to Cython, to help it compile the code better :

In [ ]:
%%cython --annotate
## cython, typing 

def f_typed( double x ):
    return x ** 2 - x


def integrate_f_typed( double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_typed(a + i * dx)
    return s * dx

In [ ]:
print("native")
%timeit -r 5 -n 3 result = integrate_f_native(0,1,1000000)
print("cython - simple")
%timeit -r 5 -n 3 result = integrate_f(0,1,1000000)
print("cython - some typing")
%timeit -r 5 -n 3 result = integrate_f_typed(0,1,1000000)

Woohoo! that's more like it.

Of course, there is more things we could do, like typing the return type of the functions and so on, as shown in this [quickstart tutorial](https://cython.readthedocs.io/en/latest/src/quickstart/cythonize.html) (which this example is grabbed from). 

<br>

These compiling tools usually won't work with external libraries, but a cool thing about Cython is that it works very well with numpy structures.

So le'ts see what we can get with our `pairwise_distance`:

In [ ]:
%%cython --annotate
import numpy as np
cimport numpy as np
cimport cython

def pairwise_distance_cython(double[:, ::1] X):
    
    cdef int num_vectors = X.shape[0]
    cdef int num_measurements = X.shape[1]
    cdef double d
    cdef double[:, ::1] D = np.empty((num_vectors, num_vectors))
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d=0
            for k in range(num_measurements):
                d += ( X[i][k] - X[j][k] )**2

            D[i, j] = d**0.5
    return(D)

In [ ]:
print(data.shape)
print('numpy:')
%timeit -n 3 -r 5 D = pairwise_distance_numpy(data)
print('cython:')
%timeit -n 3 -r 5 result = pairwise_distance_cython(data)

Okay, so now we have really gotten down.

So cython is really great, although it does take some practice to get it to work the way you want. 
They do have a [nice tutorial](https://cython.readthedocs.io/en/latest/index.html) though.

> note : cython is also a great way to interface python and C code.

> it is also fairly easy to do [profiling on cython code](https://cython.readthedocs.io/en/latest/src/tutorial/profiling_tutorial.html)

<br>
<br>

[back to the toc](#toc)


## 2.3. Numba <a id='6'></a>

**[Numba](https://numba.pydata.org/)** is an interesting alternative to Cython. It provide a number of optimization routines for python code, the most well know being **`@jit`** for **just-in-time** compilation

In [ ]:
from numba import jit

In [ ]:
# Unchanged code 

@jit
def pairwise_distance_numba(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    D = np.empty((num_vectors, num_vectors), dtype=np.float64)
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            d = 0.
            for k in range(num_measurements):
                d += np.square( np.subtract(X[i][k], X[j][k])  )
            D[i, j] = np.sqrt(d)
    return(D)

In [ ]:
%time result = pairwise_distance_numba(data)

> the first time it is executed the function is compiled. Run the function again to get the execution time without compilation

In [ ]:
%timeit -n 5 -r 3 result = pairwise_distance_numba(data)

**Woosh!** results similar to Cython, but without all the hard work of typing!

In [ ]:
# alternative syntax
import numba
pairwise_distance_numba = numba.jit(pairwise_distance_numpy)

Here it is pretty bluffing, but sometimes it can be a bit difficult to get this level of performance.

Most external libraries are missing from numba, and [not all of numpy's code has been ported as well](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html).

> Note : a lot of function in external libraries (such as the ones of sklearn) have already been optimized and compiled, so there would not necessarily be much to gain there anyway...

All-in-all, it depends quite a lot on the particulars of what you want to optimize : [here are some tips](https://numba.pydata.org/numba-doc/latest/user/performance-tips.html)


> there also exists ways to [compile numba code ahead of time](https://numba.pydata.org/numba-doc/dev/user/pycc.html)

<br>

### Comparison with replicates

In [ ]:
%timeit -n 1 -r 10 result = pairwise_distance_numpy(data)

In [ ]:
%timeit -n 1 -r 10 result = pairwise_distance_cython(data)

In [ ]:
%timeit -n 1 -r 10 result = pairwise_distance_numba(data)

> This is an example which tends to favors optimization by numba. In some other cases Cython may perform better.

<br>
<br>

[back to the toc](#toc)


# 3. Working with processes/threads <a id='7'></a>
------------------------------------------------------

## 3.1 Multiprocessing (and refactoring) <a id='8'></a>


We can take advantage of multiple cores using the `multiprocessing` module. 

In this approach, separate __processes__ are used, __not threads__. 

The use of threads is generally blocked by Python because of the *"Global Interpreter Lock"*. This was a necessary design feature as a trade-off for the enormous flexibility in memory management that Python makes possible. This means that there is no shared memory when using multiprocessing, and thus the individual tasks must be independent.

`multiprocessing` generally works well with lists, where one maps a function to each element of the list and these operations are computed as separated processes on separate cores per element of the list. To do this, we'll need to refactor our silly distance function. One approach would be to populate a list containing each of the vector pairs. The drawback here is the memory overhead of this list object:

In [ ]:
data = np.random.uniform(size=(100,100))

In [ ]:
def pairwise_list(X):

    list_of_tuples = list()
    
    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            list_of_tuples.append((X[i],X[j]))
            
    return list_of_tuples

In [ ]:
list_of_tuples = pairwise_list(data)

In [ ]:
list_of_tuples[0] #instpect 1st element

Now we'll need to refactor our function for computing distances:

In [ ]:
def pairwise_distance_rf(X):
    
    assert(len(X) == 2)
    assert(len(X[0]) == len(X[1]))
    
    d=0.
    for k in range(len(X[1])):
        d += np.square( np.subtract(X[0][k], X[1][k]))
    
    return( np.sqrt(np.sum(d) ) )

In [ ]:
%timeit -n 1 -r 3  result = list(map(pairwise_distance_rf,  list_of_tuples))

We are down to native python performance...

Let's now repreat this with multiprocessing : 

In [ ]:
import multiprocessing as mp

In [ ]:
# we use the with statement to be sure we do not forget to do a pool.close() 
with mp.Pool(2) as pool :
    %timeit -n 1 -r 3  result = pool.map(pairwise_distance_rf, list_of_tuples)

Nice! So far : **2 process ~ 1.5-2  x speedup**

Let's see if that holds up :

In [ ]:
for NP in [1,2,4,8]:
    print(NP)
    with mp.Pool(NP) as pool :
        %timeit -n 1 -r 3  result = pool.map(pairwise_distance_rf, list_of_tuples)

So what we see is fairly classical : we get diminishing return with each new process because of the overhead of multiprocessing. 
Still, the alternative was to let most of you CPU idle...

However, `multiprocessing` does not always lead to speedups as it may prevent numpy from doing some of its optimization tricks, such as vectorization.

For instance, let's see how `multiprocessing` work on the vectorized version of our function :

In [ ]:
def pairwise_distance_rf_2(X):
    
    assert(len(X) == 2)
    assert(len(X[0]) == len(X[1]))
    
    d = np.square( np.subtract(X[0], X[1]))
    
    return( np.sqrt(np.sum(d) ) )

In [ ]:
data = np.random.uniform(size=(200,100)) # I make the data a bit larger
list_of_tuples = pairwise_list(data)

In [ ]:
%timeit -n 1 -r 3  result = list(map(pairwise_distance_rf_2, list_of_tuples))

We are back to numpy performance levels

In [ ]:

for NP in [1,2,4,8]:
    print(NP)
    with mp.Pool(NP) as pool :
        %timeit -n 1 -r 3  result = pool.map(pairwise_distance_rf_2, list_of_tuples)

... no tremendous performance gains ...

<br>
<br>

[back to the toc](#toc)


## 3.2. numba and parallelization  <a id='9'></a>

It is possible to provide a `numba` function to `mp.pool`, but `numba` already provides what's necessary to parallelize your code :

In [ ]:
from numba import njit, prange

# njit -> no-python jit

@njit(parallel=True)
def pairwise_distance_numba_prange(X):

    num_vectors = X.shape[0]
    num_measurements = X.shape[1] 
    D = np.empty((num_vectors, num_vectors), dtype=np.float64)
    
    for i in prange(num_vectors): # note usage of prange
        for j in range(num_vectors):
            d = 0.
            for k in range(num_measurements):
                d += np.square( np.subtract(X[i][k], X[j][k])  )
            D[i, j] = np.sqrt(d)
    return(D)

toydata = np.random.uniform(size=(10,10)) # I make toy data to launch the function once and compile it
toyresult = pairwise_distance_numba_prange( toydata ) 

In [ ]:
print("No numba, vectorized")
%timeit -n 1 -r 3  result = list(map(pairwise_distance_rf_2, list_of_tuples))
print("numba")
%timeit -n 1 -r 3 result = pairwise_distance_numba( data )
print("numba parallel=True")
%timeit -n 1 -r 3 result = pairwise_distance_numba_prange( data )

A significant improvement with Numba again, but clearly Numba really shines when working with nested loops.


> cython also has ways to do some openMP in its compiled code.

**In the end** which of these different method you should use will depend on the particulars of the computations you have to make.